In [1]:
import pickle
import os
from ANEMO import read_edf
from ANEMO import ANEMO
import numpy as np
from lmfit import  Model, Parameters

In [2]:
def velocity_line(x, dir_target, start_anti, a_anti, latency, maxi, ramp_pursuit, do_whitening) :

    a_anti = a_anti/1000 # to switch from sec to ms
    ramp_pursuit = dir_target*(ramp_pursuit)/1000
    time = x
    vitesse = []
    y = 0
    end_ramp_pursuit = ((dir_target*maxi)/ramp_pursuit) + latency
    
    for t in range(len(time)):
        
        if start_anti >= latency :
            
            if time[t] < latency :
                vitesse.append(0)
            else :
                if latency >= end_ramp_pursuit :
                    vitesse.append(dir_target*maxi)
                else :
                    if time[t] < end_ramp_pursuit : vitesse.append((time[t]-latency)*ramp_pursuit)
                    else :                vitesse.append(dir_target*maxi)
        
        #-------------------------------------------------------------------------------
        else :
            if time[t] < start_anti :
                vitesse.append(0)
            else :
                if time[t] < latency :
                    vitesse.append((time[t]-start_anti)*a_anti)
                    y = (time[t]-start_anti)*a_anti
                else :
                    if latency >= end_ramp_pursuit :
                        vitesse.append(dir_target*maxi)
                    else :
                        if time[t] < int(end_ramp_pursuit) : vitesse.append((time[t]-latency)*ramp_pursuit+y)
                        else :                vitesse.append(dir_target*maxi+y)
        

    return vitesse

In [3]:
def velocity_sigmo(x, dir_target, start_anti, a_anti, latency, maxi, ramp_pursuit, do_whitening) :

    a_anti = a_anti/1000 # to switch from sec to ms
    ramp_pursuit = -ramp_pursuit/1000
    time = x
    vitesse = []
    y = 0
    
    e = np.exp(1)
    time_r = np.arange(-e, len(time), 1)
    
    start_rampe = dir_target*(maxi/(1+np.exp(((ramp_pursuit*time_r[0])+e))))
    
    for t in range(len(time)):
        
        if start_anti >= latency :
            
            if time[t] < latency : vitesse.append(0)
            else :                 vitesse.append(((dir_target*maxi)/(1+np.exp(((ramp_pursuit*time_r[int(time[t]-latency)])+e))))-start_rampe)
        #-------------------------------------------------------------------------------
        else :
            if time[t] < start_anti :
                vitesse.append(0)
            else :
                if time[t] < latency :
                    vitesse.append((time[t]-start_anti)*a_anti)
                    y = (time[t]-start_anti)*a_anti
                else :
                    vitesse.append(((dir_target*maxi)/(1+np.exp(((ramp_pursuit*time_r[int(time[t]-latency)])+e))))+(y-start_rampe))
        
    return vitesse

In [4]:
sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
time = ['2017-10-23_100057','2017-09-26_095637',
        '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250',
        '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313',
        '2017-11-08_150410','2017-11-17_172706']

In [5]:
time_sup = 280
opt_base = {'stop_search_misac': None, 'time_sup': 280,
            'param_fit': None, 'inde_vars': None,
            'step_fit':2, 'do_whitening': False,
            'before_sacc': 5, 'after_sacc': 15}    

In [6]:
def extraction_goodness_of_fit(equation, name) :
    for x in range(len(sujet)) :

        timeStr = time[x]
        observer = sujet[x]
        mode = 'enregistrement'

        print(observer,timeStr)

        file = os.path.join('data', mode + '_' + observer + '_' + timeStr + '.pkl')
        with open(file, 'rb') as fichier :
            param_exp = pickle.load(fichier, encoding='latin1')

        resultats = os.path.join('data', mode + '_' + observer + '_' + timeStr + '.asc')
        data = read_edf(resultats, 'TRIALID')

        A = ANEMO(param_exp)
        Fit = ANEMO.Fit(param_exp)


        N_trials = param_exp['N_trials']
        N_blocks = param_exp['N_blocks']

        goodness_of_fit = {}

        for s in ['nfev', 'residual', 'chisqr', 'redchi', 'aic', 'bic'] :
            goodness_of_fit[s] = []

        for block in range(N_blocks) :

            for s in ['nfev', 'residual', 'chisqr', 'redchi', 'aic', 'bic'] :
                goodness_of_fit[s].append([])

            for trial in range(N_trials) :

                print(block, trial)

                trial_data = trial + N_trials*block
                arg = A.arg(data[trial_data], trial=trial, block=block)
                opt = opt_base.copy()
                opt.update(arg)

                velocity_NAN = A.velocity_NAN(**opt, filt='velocity')

                data_trial = velocity_NAN[:-time_sup]
                trackertime = arg.trackertime[:-time_sup]

                max_latency = []
                for s in range(len(arg.saccades)) :
                    if (arg.saccades[s][0]-arg.t_0) >= (arg.TargetOn-arg.t_0+100) :
                        max_latency.append((arg.saccades[s][0]-arg.t_0))
                if max_latency == [] :
                    max_latency.append(len(trackertime))
                max_latency = max_latency[0]

                value_latency, value_maxi, value_anti = A.classical_method.Full(data_trial, arg.TargetOn-arg.t_0)

                if np.isnan(value_latency): value_latency= arg.TargetOn-arg.t_0+100
                if np.isnan(value_anti): value_anti= 0.
                if np.isnan(value_maxi): value_maxi= 15.


                if value_latency >= max_latency-50 : value_latency = max_latency-150
                if value_latency > 250 :             value_latency = arg.TargetOn-arg.t_0+100

                    
                if equation==velocity_sigmo :
                    param_fit=[{'name':'ramp_pursuit', 'value':40, 'min':40, 'max':80, 'vary':'vary'},
                               {'name':'maxi', 'value':value_maxi, 'min':5., 'max':40., 'vary':True},
                               {'name':'dir_target', 'value':arg.dir_target, 'min':None,
                                'max':None, 'vary':False },
                               {'name':'a_anti', 'value':value_anti, 'min':-40., 'max':40., 'vary':True  },
                               {'name':'latency', 'value':value_latency, 'min':arg.TargetOn-arg.t_0+75,
                                'max':max_latency, 'vary':True  },
                               {'name':'start_anti', 'value':arg.TargetOn-arg.t_0-100, 'min':arg.StimulusOf-arg.t_0-200,
                                'max':arg.TargetOn-arg.t_0+75, 'vary':'vary'}]

                elif equation==velocity_line :
                    param_fit=[{'name':'ramp_pursuit', 'value':100, 'min':40, 'max':800, 'vary':'vary'},
                               #{'name':'ramp_pursuit', 'value':40, 'min':40, 'max':80, 'vary':'vary'},
                               {'name':'maxi', 'value':value_maxi, 'min':5., 'max':40., 'vary':True},
                               {'name':'dir_target', 'value':arg.dir_target, 'min':None,
                                'max':None, 'vary':False },
                               {'name':'a_anti', 'value':value_anti, 'min':-40., 'max':40., 'vary':True  },
                               {'name':'latency', 'value':value_latency, 'min':arg.TargetOn-arg.t_0+75,
                                'max':max_latency, 'vary':True  },
                               {'name':'start_anti', 'value':arg.TargetOn-arg.t_0-100, 'min':arg.StimulusOf-arg.t_0-200,
                                'max':arg.TargetOn-arg.t_0+75, 'vary':'vary'}]
                
                inde_vars={'x':np.arange(len(trackertime))}

                
                model = Model(equation, independent_vars=inde_vars.keys())

                vary = False
                params = Parameters()
                for num_par in range(len(param_fit)) :
                    if param_fit[num_par]['vary'] == 'vary' : var = vary
                    else :                                    var = param_fit[num_par]['vary']
                    params.add(param_fit[num_par]['name'],
                               value=param_fit[num_par]['value'],
                               min=param_fit[num_par]['min'],
                               max=param_fit[num_par]['max'],
                               vary=var)

                params.add('do_whitening', value=False, vary=False)

                out = model.fit(data_trial, params, nan_policy='omit', **inde_vars)

                # make the other parameters vary now
                for num_par in range(len(param_fit)) :
                    if param_fit[num_par]['vary'] == 'vary' :
                        out.params[param_fit[num_par]['name']].set(vary=True)

                f = model.fit(data_trial, out.params, method='nelder', nan_policy='omit', **inde_vars)
                #-------------------------------------------------
                goodness_of_fit['nfev'][block].append(f.nfev)
                goodness_of_fit['residual'][block].append(f.residual)
                goodness_of_fit['chisqr'][block].append(f.chisqr)
                goodness_of_fit['redchi'][block].append(f.redchi)
                goodness_of_fit['aic'][block].append(f.aic)
                goodness_of_fit['bic'][block].append(f.bic)

        file = os.path.join('parametre', 'goodness_of_fit/goodness_of_fit_%s_%s.pkl'%(observer, name))
        with open(file, 'wb') as fichier:
            f = pickle.Pickler(fichier)
            f.dump(goodness_of_fit)

In [ ]:
extraction_goodness_of_fit(velocity_line, 'fct_Vline_filt_30')

In [ ]:
extraction_goodness_of_fit(velocity_sigmo, 'fct_Vsigmo_filt_30')